In [1]:
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import geopandas as gpd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Set the directory containing all raw data
rootDir = '/Users/alia/Documents/Github/DoDContractApp/Raw Data/Service Contracts'

# Loop through subfolders in directory
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    
    # If there are files in the folder
    if len(fileList)>0:
        
        for fname in fileList:
            if fname.endswith('.csv'): 
                print('\t%s' % fname)
                filepath = f'{dirName}/{fname}'
                df = pd.read_csv(filepath)
                df = df[['contract_award_unique_key',
                         'total_obligated_amount',
                         'award_base_action_date',
                         'awarding_agency_name',
                         'awarding_sub_agency_name',
                         'awarding_office_name',
                         'recipient_name',
                         'primary_place_of_performance_state_code',
                         'product_or_service_code_description',
                         'dod_claimant_program_description',
                         'type_of_contract_pricing',
                         'award_type',
                         'contract_bundling',
                         'solicitation_procedures',
                         'naics_code',
                         'naics_description',
                         'last_modified_date']]
        
                df.to_csv(f'/Users/alia/Documents/Github/DoDContractApp/Clean Data/Service Contracts/{fname}',index=False)

In [ ]:
# Set the directory containing all raw data
rootDir = '/Users/alia/Documents/Github/DoDContractApp/Clean Data/Service Contracts'

# Loop through subfolders in directory
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    
    # If there are files in the folder
    if len(fileList)>0:
        
        col_list = ['awarding_sub_agency_name',
                    'awarding_office_name',
                    'recipient_name',
                    'primary_place_of_performance_state_code',
                    'product_or_service_code_description',
                    'dod_claimant_program_description',
                    'type_of_contract_pricing',
                    'award_type',
                    'contract_bundling',
                    'solicitation_procedures',
                    'naics_description']
        
        for col in col_list:
            print(col)
            
            # Create empty dataframe
            df = pd.DataFrame()
        
            for fname in fileList:

                if fname.endswith('.csv'): 
                    print('\t%s' % fname)
                    filepath = f'{dirName}/{fname}'
                    dat = pd.read_csv(filepath)

                    sub = dat.groupby([col])[['total_obligated_amount']].sum()
                    sub.reset_index(inplace=True)
                    sub = sub.sort_values('total_obligated_amount',ascending=False)
                    sub = sub.reset_index(drop=True)
                    yr = int(fname[2:6])
                    sub['fiscal_year']=yr

                    if len(sub)>10 and col!='primary_place_of_performance_state_code':
                        # Add together groups with smaller total obligations as "Other"
                        other = sub[11:]
                        other_sum = other['total_obligated_amount'].sum()

                        sub = sub[:10]
                        sub.loc[len(sub)] = ['OTHER',other_sum,yr]
                        sub = sub.sort_values('total_obligated_amount',ascending=True)
                        sub = sub.reset_index(drop=True)

                    df = df.append(sub)

            df.to_csv(f'/Users/alia/Documents/Github/DoDContractApp/Clean Data/Plot Data/{col}.csv',index=False)

In [ ]:
# Set the directory containing all raw data
rootDir = '/Users/alia/Desktop/Service Contracts Clean'

# Loop through subfolders in directory
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    
    # If there are files in the folder
    if len(fileList)>0:
        
        col_list = ['awarding_sub_agency_name',
                    'awarding_office_name',
                    'recipient_name']
        
        for col in col_list:
            print(col)
            
            # Create empty dataframe
            df = pd.DataFrame()
        
            for fname in fileList:

                if fname.endswith('.csv'): 
                    print('\t%s' % fname)
                    filepath = f'{dirName}/{fname}'
                    dat = pd.read_csv(filepath)

                    sub = pd.DataFrame({'count' : dat.groupby([col]).size()}).reset_index()
                    sub = sub.sort_values('count',ascending=False)
                    sub = sub.reset_index(drop=True)
                    yr = int(fname[2:6])
                    sub['fiscal_year']=yr

                    if len(sub)>10:
                        # Add together groups with smaller total obligations as "Other"
                        other = sub[11:]
                        other_sum = other['count'].sum()

                        sub = sub[:10]
                        sub.loc[len(sub)] = ['OTHER',other_sum,yr]
                        sub = sub.sort_values('count',ascending=True)
                        sub = sub.reset_index(drop=True)

                    df = df.append(sub)

            df.to_csv(f'/Users/alia/Documents/Github/DoDContractApp/Clean Data/Plot Data/{col}_count.csv',index=False)